## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import path

## Compile and program project

In [5]:
project_path = [path.curdir, "AES-S"]
project_path

['.', 'AES-S']

In [6]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:4212
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00004344.
[DEBUG] Memory segment base written at 0x00004344. Size = 0x00000088.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [53]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_aes"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 5,
    "window_title": "Experiment AES",
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig

with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
#     data = dgilib.logger.log(config_dict["plot_xmax"],stop_fn)
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


In [52]:
print(data)

Interfaces:
	 256:   power,   samples:   43000
	  48:   gpio,    samples:     405



## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [13]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [14]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [15]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [16]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(encrypt_charge) - 1}")

MAX_AES_BLOCKS: 50


In [17]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [32]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

AttributeError: 'Model' object has no attribute 'params'

In [37]:
aa = results[0]

IndexError: list index out of range

In [41]:
aa.values

{'slope': 3.207061537313892e-08, 'intercept': 8.531570445904719e-10}

In [31]:
pars

Parameters([('slope', <Parameter 'slope', 0, bounds=[-inf:inf]>),
            ('intercept', <Parameter 'intercept', 1, bounds=[-inf:inf]>)])

In [19]:
fig2 = plt.figure(figsize=(9, 6))
fig2.canvas.set_window_title("Analysis AES")

In [20]:
charge_color = 'r'
time_color = 'b'

In [21]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [22]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
ax1.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} C per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} C per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [23]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(encrypt_charge, open("aes_encrypt_charge.p", "wb"))
pickle.dump(decrypt_charge, open("aes_decrypt_charge.p", "wb"))

## Write config file

In [58]:
import json

config = {}
config["name"] = "AES"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["AES Encrypt"], 3: ["AES Decrypt"]},
                      "result_types": ["Charge", "Time"],
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": MBEDTLS_AES_BLOCK_SIZE}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)